In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

import os
import re
from tqdm.notebook import tqdm

# Summary
In this notebook I carry out EDA, cleaning and processing of data.

* tournaments_male.csv. Decided this is not useful so did nothing with this
* matches_male.csv.
    * Dropped the columns `tournament_index` and `round`
    * Categorized the `result` column and extracted the score in games from this
    * Dropped all entries in which the result is 'bye' or one of two outlier matches
    * Extracted winners' and losers' names, countries and seeds

### Load data

In [ ]:
dir_raw = "../data/raw/"
dir_processed = "../data/processed"

In [ ]:
os.listdir(dir_raw)

In [ ]:
matches = pd.read_csv(dir_raw + "matches_male.csv", index_col=0)
tournaments = pd.read_csv(dir_raw + "tournaments_male.csv", index_col=0)

In [ ]:
matches.head(5)

In [ ]:
tournaments.head(5)

For the aim of the project, this tournament dataframe is not necessary so I will not explore it further

### `tournament_index` and `round`

In [ ]:
# drop tournament index as we do not need tournament information for elo
matches = matches.drop(columns=["tournament_index"])

In [ ]:
matches["round"].value_counts()

In [ ]:
matches = matches.drop(columns=["round"])

We can see that the round column is a little bit dirty. However, like tournament index, it is actually not useful for us so I decided to drop it.

### `result`, extracting information from results column

The next few functions took several iterations and experimentation to create. I did not record the process by which I incrementally improved the function.

In [ ]:
def determine_result_type(result: str) -> (str):
    if "bye" in result:
        return "bye"

    if "w/o" in result:
        return "wo"

    if "ret" in result:
        return "ret"

    if "unknown" in result:
        return "unknown"

    if result == "No shows":
        return "other"

    if result == "Final not played due to unsafe court conditions":
        return "other"

    if "3/" in result:
        # then result is of the form "3/i" or "3/i (15m)" where i=0,1 or 2
        return "3/"

    if "2/" in result:
        # then result is of the form "2/i" or "2/i (15m)" where i=0,1
        return "2/"

    # pattern should match '11-3, 4-11, 11-9 (44m)'
    pat = r"(?P<points>[\d, -]+\d)(?: \((?P<time>\d+)m\))?"
    if re.match(pat, result):
        return "points"

    # if none of the above conditions were met, then
    # want to see what it is
    if True:
        print(result)

In [ ]:
# manual testing of determine_result_type.
# If nothing is printed then presumption is that
# function is working
for result in matches.result.to_list():
    determine_result_type(result)

In [ ]:
def determine_game_score_from_points(points: str) -> (int, int):
    """
    Determine the score in games of a match.

    Parameters
    ----------
    points : str
        The points in each game of a match. Should be something
        like '11-8, 7-11, 11-9, 12-10'
        * points for each game separated by ', '
        * points of players in a single game separated by '-'
        * in each game, the first number is the points of the
        winner of the match

    Returns
    -------
    n_games_won_by_winner : int
        The number of games won by the winner of the match

    n_games_won_by_loser : int
        The number of games won by the loser of the match
    """
    # parse the input string
    points_parsed = [[int(i) for i in point.split("-")] for point in points.split(", ")]

    # calculate number of games won by winner and loser.
    # convention is that for each game, the first number
    # in the pair is the points of the winner of the match.
    n_games_won_by_winner = 0
    n_games_won_by_loser = 0
    for points_winner, points_loser in points_parsed:
        if points_winner > points_loser:
            n_games_won_by_winner += 1
        elif points_loser > points_winner:
            n_games_won_by_loser += 1

    return n_games_won_by_winner, n_games_won_by_loser

In [ ]:
# manual testing of determine_game_score
print(determine_game_score_from_points("11-8, 7-11, 11-9, 12-10"))
print(determine_game_score_from_points("1-11, 11-3, 11-9"))

In [ ]:
def process_results_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Process raw results column to get games won by winner and loser
    
    Parameters
    ----------
    df : pd.DataFrame
        dataframe containing a column 'result'
    
    Returns
    -------
    pd.DataFrame
        New dataframe that is copy of the original but with two
        additional columns, for the number of games won by
        the winner and loser respectively. 
    """
    df_copy = df.copy()
    
    games_winner = []
    games_loser = []
    results = df_copy.result.to_list()

    for result in tqdm(results):
        result_type = determine_result_type(result)
        
        if result_type in ['bye', 'wo', 'ret', 'unknown', 'other']:
            games_winner.append(np.nan)
            games_loser.append(np.nan)
        
        elif result_type in ['3/', '2/']:
            # then result is of the form "3/i ..." or "2/i ..."
            games_winner.append(int(result[0]))
            games_loser.append(int(result[2]))
        
        elif result_type == 'points':
            # we presume results is of standard form
            # "11-5, 11-9, 12-10 ..."
            pat = r"(?P<points>[\d, -]+\d)(?: \((?P<time>\d+)m\))?"
            match = re.match(pat, result)
            points = match.group('points')
            w, l = determine_game_score_from_points(points)
            
            games_winner.append(w)
            games_loser.append(l)
        
        else:
            # if functions created properly, this should never run
            print(result)
            print(result_type)

    df_copy['games_winner'] = games_winner
    df_copy['games_loser'] = games_loser
    
    return df_copy

In [ ]:
matches = process_results_column(matches)
matches

## `players`, extracting information

In [ ]:
for _, row in matches.iterrows():
    if "bt" not in row.players and row.result != "bye":
        print(row)

Based on above, want to drop the entries which are bye or these other outlier events. All other entries will have 'bt' in players, so there was a winner and a loser

In [ ]:
# remove entries in which there was no winner or loser
bad_results = ["bye", "No shows", "Final not played due to unsafe court conditions"]
indices = ~matches.result.isin(bad_results)
matches = matches[indices]

In [ ]:
# check that all entries of players have 'bt'
for _, row in matches.iterrows():
    if "bt" not in row.players:
        print(row)

In [ ]:
def split_players_to_winner_loser(df):
    """
    Split players column in raw data into winner and loser.

    Parameters
    ----------
    df : pd.DataFrame
        dataframe containing a column 'players' where all
        entries are strings of the form '{winner} bt {loser}'

    Returns
    -------
    pd.DataFrame
        New dataframe that is copy of the original but with two
        additional columns, one for winner info and one for loser
        info
    """
    df_new = df.copy()

    players = df_new.players.str.split(pat=" bt ")
    df_new["winner"] = players.map(lambda x: x[0])
    df_new["loser"] = players.map(lambda x: x[1])
    return df_new

In [ ]:
matches = split_players_to_winner_loser(matches)
matches

In [ ]:
def parse_player_entry(player: str) -> (str, str, str):
    """
    Extract player name, country and seed from player entry.
    
    Parameter
    ---------
    player: str
        String of the form '[9/16] Amaad Fareed (PAK)'
    
    Returns
    -------
    (str, str, str)
        player's name, player's country, player's seed
    """
    pat_seed = r"(\[(?P<seed>\S+)\] )?"
    pat_name = r"(?P<name>(?:\w+[ '-])+\w+)"
    pat_bracketed_info = r"(?: \(\w+\))?"
    pat_country = r" \((?P<country>[A-Z]{3,3})\)"
    pat = pat_seed+pat_name+pat_bracketed_info+pat_country
    
    match = re.search(pat, player)
    
    try:
        seed = match.group('seed')
        name = match.group('name')
        country = match.group('country')
        return (name, country, seed)
    except:
        print(player)
        return (np.nan, np.nan, np.nan)

In [ ]:
def split_player_info(df, column: str) -> pd.DataFrame:
    """
    Split the winner or loser column into seed, name and country.
    
    Parameters
    ----------
    df: pd.DataFrame
        dataframe with winner or loser column, as extracted
        from the function `split_players_to_winner_loser`
    columns: str
        either 'winner' or 'loser'
    
    Returns
    -------
    pd.DataFrame
        new dataframe with three additional columns:
        * {column}_name
        * {column}_country
        * {column}_seed
    """
    df_copy = df.copy()
    player_info = df_copy[column].to_list()
    names = []
    countries = []
    seeds = []
    
    for player in player_info:
        name, country, seed = parse_player_entry(player)
        names.append(name)
        countries.append(country)
        seeds.append(seed)
    
    df_copy[f"{column}_name"] = names
    df_copy[f"{column}_country"] = countries
    df_copy[f"{column}_seed"] = seeds
    
    return df_copy

In [ ]:
matches = split_player_info(matches, 'loser')
matches = split_player_info(matches, 'winner')
matches

## Reorder so it is chronological

In [ ]:
matches = matches[::-1].reset_index(drop=True)

## Save the dataframe

In [ ]:
matches.to_csv(dir_processed+"matches.csv")

In [ ]:
test = pd.read_csv(dir_processed+"matches.csv", index_col=0)
test